In [ ]:
import json
import pandas as pd
import os
import numpy as np
import re

## Full Dataset Analysis

In [ ]:
dataset_path = os.getcwd() + '/..' + '/data/full_dataset.csv'
dataset_path

In [ ]:
df = pd.read_csv(dataset_path)

In [ ]:
print(df.columns)
print(df.shape)

In [ ]:
df = df.dropna()

In [ ]:
df["title"].apply(type).value_counts()

In [ ]:
df["ingredients"].apply(type).value_counts()

In [ ]:
df["directions"].apply(type).value_counts()

In [ ]:
df[df['title'].apply(type) == float]  ## Helped to find a title that was a NaN

In [ ]:
len(eval(df.ingredients.loc[0])) > 2

In [ ]:
remove1 = df.loc[df.title.map(lambda x: len(x) < 5)]
remove2 = df.loc[df.ingredients.map(lambda x: len(eval(x)) < 3)]
remove3 = df.loc[df.directions.map(lambda x: len(eval(x)) < 2 or len(x) < 30)]
remove4 = df.loc[df.directions.map(lambda x: re.search('(step|mix all)', ''.join(x), re.IGNORECASE) != None)]

In [ ]:
print('remove1:', len(remove1))
print('remove2:', len(remove2))
print('remove3:', len(remove3))
print('remove4:', len(remove4))

In [ ]:
print('len of removed lines: ' + str(len(remove3)+len(remove2)+len(remove1)+len(remove4)))

## Json Data Analysis

In [ ]:
file = os.getcwd() + '/..' + '/data/json_recipes_500000.json'
file

In [ ]:
df = pd.read_json(file)

In [ ]:
df

In [ ]:
df['input'].value_counts()

In [ ]:
df['input'].

In [ ]:
def filter_txt(input_path, output_path, logger):
    logger.info("Filtering "+ input_path + " to " + output_path)
    count = 0
    bad_lines = pd.DataFrame()
    pattern1 = r"<RECIPE_START>"
    pattern2 = r"<RECIPE_END>"
    with open(input_path, 'r') as f_in:
        with open(output_path, 'w') as f_out:
            for i, row in tqdm(enumerate(f_in), desc="Filtering"):
                if re.search(pattern1, row) and re.search(pattern2, row):
                    ingr_start_index = row.find("<INPUT_START>")
                    ingr_end_index = row.find("<INPUT_END>")
                    if ingr_end_index - ingr_start_index > 10 and ingr_end_index - ingr_start_index < 300:
                        f_out.write("{}\n".format(row))
                        continue
                else:
                    d = {'index': i,
                         'txt': row}
                    new_row = pd.DataFrame(d, index=[0])
                    bad_lines = pd.concat([bad_lines, new_row]).reset_index(drop=True)
                    count += 1